In [1]:
import os
os.chdir('../')


In [2]:
%pwd

'd:\\MLOps-Project\\Kidney-disease-classification-mlops'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    training_model_path : Path
    updata_base_model_path  :  Path
    training_data: Path
    params_epochs : int
    params_is_augmentation : bool
    params_batch_size : int
    params_image_size : list
 

In [9]:
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constant import *

from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constant import *
# Configuration
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.atifacts_root])
        
    def get_training_config(self) -> TrainingConfig:
        training =  self.config.training
        prepare_base_model =self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, 'kidney-ct-scan-image') 
        
        create_directories([
            Path(training.root_dir)
        ])
        
        training_config = TrainingConfig(
        root_dir= Path(training.root_dir),
        training_model_path=Path(training.trained_model_path),
        updata_base_model_path=Path(prepare_base_model.updated_base_model_path),
        training_data = Path(training_data),
        params_epochs = params.EPOCHS, 
        params_batch_size= params.BATCH_SIZE,
        params_is_augmentation= params.AUGMENTATION,
        params_image_size= params.IMAGE_SIZE
        )
        
        return training_config
    

In [10]:
import tensorflow as tf

In [19]:
class Training:
    def __init__(self, confg : TrainingConfig):
        self.config = confg
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updata_base_model_path
        )
        
        
    def train_vaid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1 / 255,
            validation_split = 0.20
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        valid_datagernerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagernerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = True,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                
            
            rotation_range = 40,
            horizontal_flip = True,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            shear_range = 0.2,
            zoom_range = 0.2,
            **datagenerator_kwargs
            )
            
            
        else:
            train_datagenerator = valid_datagernerator
        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'training',
            shuffle = True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    def train(self):
        self.steps_per_epchs = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epchs,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )
        
        self.save_model(
            path = self.config.training_data,
            model = self.model
        )

        

In [20]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(confg=training_config)
    training.get_base_model()
    training.train_vaid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-07-31 20:16:53,704: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-31 20:16:53,707: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-31 20:16:53,709: INFO: common: Created directory  at: artifacts]
[2024-07-31 20:16:53,711: INFO: common: Created directory  at: artifacts\training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/10
[2024-07-31 20:16:55,760: WARNING: module_wrapper: From c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
23/23 [==============================] - 32s 1s/step - loss: 0.6976 - accuracy: 0.5983 - val_loss: 0.5528 - val_accuracy: 0.6750
Epoch 2/10
23/23 [==============================] - 18s 776ms/step - loss: 0.5961 - accuracy: 0.7022 - val_loss: 0.5576 - val_accuracy: 0.8250
Epoch 3/10
23/23 [=============